
| EDA dataframe      | Description |due|
| ----------- | ----------- |----------- |
| grape_train  | 캠벨얼리 가격/거래량 + 소비자 물가 지수 + 자동차용경유 + 날씨(4개-평균기온, 평균온량, 일강수량, 일교차) + 수출입(4개) + 재배면적 + 샤인머스켓 거래량 + 실업률_경제활동인구(2개-경제활동인구, 실업률) + 최저시급    |20190101 ~ 20201104|
| shinemuscat_train   |샤인마스캇  가격/거래량 + 소비자 물가 지수 + 자동차용경유 + 날씨(4개-평균기온, 평균온량, 일강수량, 일교차) + 수출입(4개) + 재배면적 + 블루베리 거래량 + 실업률_경제활동인구(2개-경제활동인구, 실업률) + 최저시급        |20190101 ~ 20201104|
| df      | grape_train + shinemuscat_train       | 20190101 ~ 2020928|

In [1]:
import matplotlib.pyplot as plt
plt.rcParams['font.family'] = 'AppleGothic'
# import tensorflow as tf
# import keras
from numpy import array
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import math
import matplotlib as mpl
from IPython.core.display import display, HTML
from datetime import datetime
from datetime import timedelta
from dateutil.relativedelta import relativedelta
from sklearn.preprocessing import MinMaxScaler, RobustScaler
# from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
# from tensorflow.keras import Sequential
# from tensorflow.keras import layers
# from tensorflow.keras.layers import Dense, LSTM, Dropout
# from keras.callbacks import ModelCheckpoint, EarlyStopping
# from statsmodels.tsa.seasonal import seasonal_decompose, STL
import urllib.request
import requests
import json
import datetime
import os
from datetime import datetime
from sklearn.ensemble import RandomForestRegressor
import seaborn as sns
from tqdm import tqdm

import warnings
warnings.filterwarnings('ignore')

/var/folders/08/yfq1rzmx605_pllcn65f77540000gn/T/ipykernel_10202/238796418.py:11: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import display, HTML


# 0. 데이터셋(dataset)

**train** 20160101 ~ 20200928 <br> 
**test** 20200929 ~ 20201105

EDA와 기본 모델을 만들때 사용 <br>
train + test 합친 프레임(토마토, 포도) = df

In [2]:
# Macbook
raw_path = '/Users/goeun/Documents/YearDream/ProJect/kurly_project/products_price_predict/data/raw/'
processed_path = '/Users/goeun/Documents/YearDream/ProJect/kurly_project/products_price_predict/data/processed/'

In [3]:
# iMACA
# raw_path = '/Users/cge/Documents/yeardream/프로젝트/기업프로젝트/products_price_predict/data/raw/'
# processed_path = '/Users/cge/Documents/yeardream/프로젝트/기업프로젝트/products_price_predict/data/processed/'

In [3]:
# test 데이터 합치기
arr = os.listdir('/Users/goeun/Documents/YearDream/ProJect/kurly_project/data/public_data/test_files')
print(arr)
print()

filename_list = arr
filename_list_2020 = []    
for filename in filename_list :
    if '2020' in filename :
        filename_list_2020.append(filename)
        
path = '/Users/goeun/Documents/YearDream/ProJect/kurly_project/data/public_data/test_files/'
df_list = []
for filename in tqdm(filename_list_2020) :
    df = pd.read_csv(path + filename)
    df_list.append(df)
test = pd.concat(df_list, sort=False).reset_index(drop=True)

['test_2020-10-12.csv', 'test_2020-10-06.csv', 'test_2020-10-07.csv', 'test_2020-10-13.csv', 'test_2020-09-30.csv', 'test.csv', 'test_2020-10-05.csv', 'test_2020-10-11.csv', 'test_2020-10-10.csv', 'test_2020-10-04.csv', 'test_2020-10-14.csv', 'test_2020-10-28.csv', 'test_2020-10-29.csv', 'test_2020-10-15.csv', 'test_2020-10-01.csv', 'test_2020-10-17.csv', 'test_2020-10-03.csv', 'test_2020-10-02.csv', 'test_2020-10-16.csv', 'test_2020-10-27.csv', 'test_2020-10-26.csv', 'test_2020-10-18.csv', 'test_2020-10-24.csv', 'test_2020-10-30.csv', 'test_2020-10-31.csv', 'test_2020-10-25.csv', 'test_2020-10-19.csv', 'test_2020-10-21.csv', 'test_2020-10-09.csv', 'test_2020-10-08.csv', 'test_2020-10-20.csv', 'test_2020-09-29.csv', 'test_2020-10-22.csv', 'test_2020-10-23.csv', '.ipynb_checkpoints', 'test_2020-11-04.csv', 'test_2020-11-05.csv', 'test_2020-11-02.csv', 'test_2020-11-03.csv', 'test_2020-11-01.csv']



100%|██████████████████████████████████████████| 38/38 [00:00<00:00, 768.65it/s]


In [4]:
arr2 = os.listdir(processed_path)
arr2

['shinemuscat_weather.csv',
 '.DS_Store',
 'shinemuscat.csv',
 'README.md',
 'grape.csv',
 'grape_실업률추가.csv',
 'grape_weather.csv',
 'shinemuscat_실업률추가.csv']

In [6]:
grape = pd.read_csv(processed_path+'grape_실업률추가.csv')
shinemuscat = pd.read_csv(processed_path+'shinemuscat_실업률추가.csv')

# train -> 2020년 9월 28일까지만 EDA 진행
grape_train = grape[:1733]
shinemuscat_train = shinemuscat[:1733]

df = pd.merge(grape_train, shinemuscat_train, on = ['date'])

print(grape_train.shape)
print(shinemuscat_train.shape)
print(df.shape)

(1733, 19)
(1733, 19)
(1733, 37)


In [7]:
print(f'캠벨얼리')
print(grape_train.info())
print()
print(f'샤인머스켓')
print(shinemuscat_train.info())

캠벨얼리
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1733 entries, 0 to 1732
Data columns (total 19 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   date           1733 non-null   object 
 1   요일             1733 non-null   object 
 2   캠벨얼리_거래량(kg)   1733 non-null   float64
 3   캠벨얼리_가격(원/kg)  1733 non-null   float64
 4   소비자물가지수        1733 non-null   float64
 5   자동차용경유         1733 non-null   float64
 6   재배지역_평균기온      1733 non-null   float64
 7   재배지역_평균운량      1733 non-null   float64
 8   재배지역_일강수량      1733 non-null   float64
 9   재배지역_일교차       1733 non-null   float64
 10  샤인마스캇_거래량(kg)  1733 non-null   float64
 11  수출(중량)         1733 non-null   float64
 12  수출(금액)         1733 non-null   float64
 13  수입(중량)         1733 non-null   float64
 14  수입(금액)         1733 non-null   float64
 15  재배면적(ha)       1733 non-null   float64
 16  최저시급           1733 non-null   float64
 17  경제활동인구         1733 non-null   int64  
 18  실업률

# Data Postproceesing (df = train, test)
        
- [ ] **후처리 -> 일요일(공휴일)의 경우 거래가 발생하지 않음 -> 모델 성능 저하 -> 전처리 필요(거래 전, 후의 가격 평균 대체)**

In [152]:
grape2 = grape.copy()

for x in range(1733):
    if x == 0:
        grape2['캠벨얼리_거래량(kg)'].iloc[x] = grape2['캠벨얼리_거래량(kg)'].iloc[x + 1]
        grape2['캠벨얼리_가격(원/kg)'].iloc[x] = grape2['캠벨얼리_가격(원/kg)'].iloc[x + 1]
    elif grape2['캠벨얼리_거래량(kg)'].iloc[x] == 0:
        grape2['캠벨얼리_거래량(kg)'].iloc[x] = grape2['캠벨얼리_거래량(kg)'].iloc[x - 1]
        grape2['캠벨얼리_가격(원/kg)'].iloc[x] = grape2['캠벨얼리_가격(원/kg)'].iloc[x - 1]

In [153]:
grape2['캠벨얼리_거래량(kg)'].value_counts()

934.8       18
738.0       12
4805.2       9
0.0          8
5725.0       7
            ..
146753.6     1
167688.1     1
177249.4     1
202000.9     1
26930.0      1
Name: 캠벨얼리_거래량(kg), Length: 1370, dtype: int64

In [179]:
shinemuscat2 = shinemuscat.copy()

for x in range(1733):
    if x == 0:
        shinemuscat2['샤인마스캇_거래량(kg)'].iloc[x] = shinemuscat2['샤인마스캇_거래량(kg)'].iloc[x + 1]
        shinemuscat2['샤인마스캇_가격(원/kg)'].iloc[x] = shinemuscat2['샤인마스캇_가격(원/kg)'].iloc[x + 1]
    elif shinemuscat2['샤인마스캇_거래량(kg)'].iloc[x] == 0:
        shinemuscat2['샤인마스캇_거래량(kg)'].iloc[x] = shinemuscat2['샤인마스캇_거래량(kg)'].iloc[x - 1]
        shinemuscat2['샤인마스캇_가격(원/kg)'].iloc[x] = shinemuscat2['샤인마스캇_가격(원/kg)'].iloc[x - 1]

In [180]:
shinemuscat2['샤인마스캇_거래량(kg)'].value_counts()

0.0        169
116.0      167
164.0      100
138.0       47
38.0        46
          ... 
8596.0       1
6542.0       1
6088.0       1
8098.0       1
74970.5      1
Name: 샤인마스캇_거래량(kg), Length: 850, dtype: int64

In [181]:
shinemuscat2[shinemuscat2['샤인마스캇_거래량(kg)'] == 0][-10:]

,date,요일,샤인마스캇_거래량(kg),샤인마스캇_가격(원/kg),소비자물가지수,자동차용경유,재배지역_평균기온,재배지역_평균운량,재배지역_일강수량,재배지역_일교차,수출(중량),수출(금액),수입(중량),수입(금액),재배면적(ha),블루베리_거래물량(kg),최저시급,경제활동인구,실업률 (%)
159,2016-06-08,수요일,0.0,0.0,95.611,1219.88,21.51,0.00,3.78,5.87,825.180000,5990.400000,60499.823333,123424.900000,14946.0,55.34,6030,955862,0.12
160,2016-06-09,목요일,0.0,0.0,95.611,1221.61,23.30,0.00,0.00,11.69,825.180000,5990.400000,60499.823333,123424.900000,14946.0,55.34,6030,955862,0.12
1735,2020-10-01,목요일,0.0,0.0,100.180,1142.56,18.45,4.83,0.00,12.30,8785.570968,133799.032258,199894.774194,600716.258065,13183.0,114.16,8590,969517,0.13
1736,2020-10-02,금요일,0.0,0.0,100.180,1142.29,19.59,6.50,0.00,11.52,8785.570968,133799.032258,199894.774194,600716.258065,13183.0,114.16,8590,969517,0.13
1737,2020-10-03,토요일,0.0,0.0,100.180,1141.54,20.08,9.29,0.72,8.58,8785.570968,133799.032258,199894.774194,600716.258065,13183.0,114.16,8590,969517,0.13
1738,2020-10-04,일요일,0.0,0.0,100.180,1141.05,19.03,9.50,0.13,8.02,8785.570968,133799.032258,199894.774194,600716.258065,13183.0,114.16,8590,969517,0.13
1745,2020-10-11,일요일,0.0,0.0,100.180,1136.39,15.35,7.03,0.00,11.81,8785.570968,133799.032258,199894.774194,600716.258065,13183.0,114.16,8590,969517,0.13
1752,2020-10-18,일요일,0.0,0.0,100.180,1133.01,11.95,1.99,0.00,16.20,8785.570968,133799.032258,199894.774194,600716.258065,13183.0,114.16,8590,969517,0.13
1759,2020-10-25,일요일,0.0,0.0,100.180,1129.46,13.06,3.06,0.00,14.18,8785.570968,133799.032258,199894.774194,600716.258065,13183.0,114.16,8590,969517,0.13
1766,2020-11-01,일요일,0.0,0.0,100.090,1124.51,11.45,9.91,2.53,3.41,1706.212667,28741.315333,22755.312333,67632.875333,13183.0,117.96,8590,972689,0.12


In [ ]:
grape2 = grape.copy()

# 일요일에만 해당되는 날짜 결측치로 만들기
grape2.loc[grape2['요일'] == '일요일' , ['캠벨얼리_거래량(kg)']] = np.nan
grape2.loc[grape2['요일'] == '일요일' , ['캠벨얼리_가격(원/kg)']] = np.nan
grape2['캠벨얼리_거래량(kg)'].fillna(method='ffill', inplace=True)
grape2['캠벨얼리_가격(원/kg)'].fillna(method='ffill', inplace=True)

In [157]:
grape2['캠벨얼리_가격(원/kg)'].value_counts()

0.0       81
3415.0     6
3484.0     5
2059.0     4
3316.0     4
          ..
1791.0     1
1669.0     1
1729.0     1
2027.0     1
3153.0     1
Name: 캠벨얼리_가격(원/kg), Length: 1229, dtype: int64

In [182]:
shinemuscat2 = shinemuscat.copy()

# 일요일에만 해당되는 날짜 결측치로 만들기
shinemuscat2.loc[grape2['요일'] == '일요일' , ['샤인마스캇_거래량(kg)']] = np.nan
shinemuscat2.loc[grape2['요일'] == '일요일' , ['샤인마스캇_가격(원/kg)']] = np.nan
shinemuscat2['샤인마스캇_거래량(kg)'].fillna(method='ffill', inplace=True)
shinemuscat2['샤인마스캇_가격(원/kg)'].fillna(method='ffill', inplace=True)

In [183]:
shinemuscat2['샤인마스캇_거래량(kg)'].value_counts()

0.0        669
200.0        6
160.0        4
208.0        4
872.0        4
          ... 
4558.0       1
2782.0       1
4276.0       1
5526.0       1
74970.5      1
Name: 샤인마스캇_거래량(kg), Length: 845, dtype: int64

In [184]:
shinemuscat2[shinemuscat2['샤인마스캇_거래량(kg)'] == 0][-10:]

,date,요일,샤인마스캇_거래량(kg),샤인마스캇_가격(원/kg),소비자물가지수,자동차용경유,재배지역_평균기온,재배지역_평균운량,재배지역_일강수량,재배지역_일교차,수출(중량),수출(금액),수입(중량),수입(금액),재배면적(ha),블루베리_거래물량(kg),최저시급,경제활동인구,실업률 (%)
1607,2020-05-26,화요일,0.0,0.0,99.44,1068.51,20.64,7.06,0.00,12.03,379.858065,3603.451613,126593.948387,341222.322581,13183.0,114.16,8590,972724,0.16
1608,2020-05-27,수요일,0.0,0.0,99.44,1070.81,19.94,6.15,0.04,11.05,379.858065,3603.451613,126593.948387,341222.322581,13183.0,114.16,8590,972724,0.16
1609,2020-05-28,목요일,0.0,0.0,99.44,1073.05,20.21,2.67,0.00,15.47,379.858065,3603.451613,126593.948387,341222.322581,13183.0,114.16,8590,972724,0.16
1617,2020-06-05,금요일,0.0,0.0,99.71,1094.27,25.87,5.07,0.00,13.20,1351.923333,19661.166667,44707.270000,97106.366667,13183.0,117.96,8590,975275,0.15
1618,2020-06-06,토요일,0.0,0.0,99.71,1099.12,22.43,7.00,0.71,10.18,1351.923333,19661.166667,44707.270000,97106.366667,13183.0,117.96,8590,975275,0.15
1619,2020-06-07,일요일,0.0,0.0,99.71,1101.38,23.27,3.03,0.02,14.78,1351.923333,19661.166667,44707.270000,97106.366667,13183.0,117.96,8590,975275,0.15
1735,2020-10-01,목요일,0.0,0.0,100.18,1142.56,18.45,4.83,0.00,12.30,8785.570968,133799.032258,199894.774194,600716.258065,13183.0,114.16,8590,969517,0.13
1736,2020-10-02,금요일,0.0,0.0,100.18,1142.29,19.59,6.50,0.00,11.52,8785.570968,133799.032258,199894.774194,600716.258065,13183.0,114.16,8590,969517,0.13
1737,2020-10-03,토요일,0.0,0.0,100.18,1141.54,20.08,9.29,0.72,8.58,8785.570968,133799.032258,199894.774194,600716.258065,13183.0,114.16,8590,969517,0.13
1738,2020-10-04,일요일,0.0,0.0,100.18,1141.05,19.03,9.50,0.13,8.02,8785.570968,133799.032258,199894.774194,600716.258065,13183.0,114.16,8590,969517,0.13
